In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
import csv
import math
from operator import itemgetter
import time

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.externals import joblib
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, scale

In [3]:
data = pd.read_csv('final.csv')
data.head()

,Male1,AGE,US_Army)DMQMILIZ,BornInUS,Citizenship(DMDCITZN),EduLevelAdults(DMDEDUC2),MaritalStatus(DMDMARTL),(Eng/Span(SIALANG),#PPLinHousehold(DMDHHSIZ),#Kinds<=5yrs(DMDHHSZA),...,CurrentSelfReportedWeight(pounds)WHD020,AgeWhenHeaviestWeight(WHQ150),HowOftenDrankMilk,DepressionScore,Race(RIDRETH3)_Asian,Race(RIDRETH3)_Black,Race(RIDRETH3)_MexicanAmerican,Race(RIDRETH3)_Other&MultiRacial,Race(RIDRETH3)_OtherHispanic,Race(RIDRETH3)_White
0,1,69,1,1,1,3,0,1,3,0,...,180,62,8,1,0,1,0,0,0,0
1,1,54,0,1,1,3,1,1,4,0,...,200,25,9,2,0,0,0,0,0,1
2,1,72,1,1,1,4,1,1,2,0,...,195,35,9,0,0,0,0,0,0,1
3,0,73,0,1,1,5,1,1,2,0,...,120,60,9,9,0,0,0,0,0,1
4,1,56,1,1,1,4,0,1,1,0,...,235,54,9,19,0,0,1,0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6065 entries, 0 to 6064
Data columns (total 63 columns):
Male1                                             6065 non-null int64
AGE                                               6065 non-null int64
US_Army)DMQMILIZ                                  6065 non-null int64
BornInUS                                          6065 non-null int64
Citizenship(DMDCITZN)                             6065 non-null int64
EduLevelAdults(DMDEDUC2)                          6065 non-null int64
MaritalStatus(DMDMARTL)                           6065 non-null int64
(Eng/Span(SIALANG)                                6065 non-null int64
#PPLinHousehold(DMDHHSIZ)                         6065 non-null int64
#Kinds<=5yrs(DMDHHSZA)                            6065 non-null int64
#ofKinds6-17yrs(DMDHHSZB)                         6065 non-null int64
#ofAdults>=60yrs(DMDHHSZE)                        6065 non-null int64
HouseholdIncome(INDHHIN2)                         606

In [6]:
#data['$spentAtSupermarket/grocery store(CBD070)'].value_counts()

In [33]:
replace_grocery = data.replace({'$spentAtSupermarket/grocery store(CBD070)': {777777: 2000, 999999: 2000}})
#replace_grocery['$spentAtSupermarket/grocery store(CBD070)'].value_counts()

In [34]:
replace_non_food = replace_grocery.replace({'#MealsFastFoodOrPizzaPlace(DBD900)': {777777: 1000, 999999: 1000}})

In [35]:
from sklearn import preprocessing

In [36]:
replace_non_food['log_vitaminA'] = np.log(replace_non_food['Vitamin A,RAE(mcg)DR1TVARA'])


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [38]:
replace_non_food['Energy(kcal)DR1TKCAL'] = np.log(replace_non_food['Energy(kcal)DR1TKCAL'])

In [41]:
#replace_non_food['Energy(kcal)DR1TKCAL']

In [42]:
replace_non_food[' Calcium(mg)DR1TCALC'] = np.log(replace_non_food[' Calcium(mg)DR1TCALC'])

In [43]:
replace_non_food['Sodium(mg)DR1TSODI'] = np.log(replace_non_food['Sodium(mg)DR1TSODI'])

In [44]:
replace_non_food['WaterDrankYesterday(gm)DR1.320Z'] = np.log(replace_non_food['WaterDrankYesterday(gm)DR1.320Z'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [46]:
#replace_non_food['WaterDrankYesterday(gm)DR1.320Z']

In [47]:
replace_non_food['VitaminB12(pmol/L)LBDB12SI'] = np.log(replace_non_food['VitaminB12(pmol/L)LBDB12SI'])

In [49]:
#replace_non_food['VitaminB12(pmol/L)LBDB12SI']

In [53]:
replace_non_food.Male1.value_counts()

0    3167
1    2898
Name: Male1, dtype: int64

In [55]:
replace_gender = replace_non_food.replace({'Male1': {0: 'female', 1: 'male'}})
replace_gender.Male1.value_counts()

female    3167
male      2898
Name: Male1, dtype: int64

In [57]:
NewData = pd.get_dummies(replace_gender)

In [64]:
New = NewData.rename(index=str, columns={"Male1_female": "female", "Male1_male": "male"})
New.head()

,AGE,US_Army)DMQMILIZ,BornInUS,Citizenship(DMDCITZN),EduLevelAdults(DMDEDUC2),MaritalStatus(DMDMARTL),(Eng/Span(SIALANG),#PPLinHousehold(DMDHHSIZ),#Kinds<=5yrs(DMDHHSZA),#ofKinds6-17yrs(DMDHHSZB),...,DepressionScore,Race(RIDRETH3)_Asian,Race(RIDRETH3)_Black,Race(RIDRETH3)_MexicanAmerican,Race(RIDRETH3)_Other&MultiRacial,Race(RIDRETH3)_OtherHispanic,Race(RIDRETH3)_White,log_vitaminA,female,male
0,69,1,1,1,3,0,1,3,0,0,...,1,0,1,0,0,0,0,4.700480,0,1
1,54,0,1,1,3,1,1,4,0,2,...,2,0,0,0,0,0,1,7.305188,0,1
2,72,1,1,1,4,1,1,2,0,0,...,0,0,0,0,0,0,1,6.484635,0,1
3,73,0,1,1,5,1,1,2,0,0,...,9,0,0,0,0,0,1,6.760415,1,0
4,56,1,1,1,4,0,1,1,0,0,...,19,0,0,1,0,0,0,5.872118,0,1


In [70]:
dropped = New.drop(columns=['Vitamin A,RAE(mcg)DR1TVARA'])

In [73]:
cols = list(dropped)
cols.insert(0, cols.pop(cols.index('DepressionScore')))
#cols

In [75]:
df = dropped.ix[:, cols]
df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,DepressionScore,AGE,US_Army)DMQMILIZ,BornInUS,Citizenship(DMDCITZN),EduLevelAdults(DMDEDUC2),MaritalStatus(DMDMARTL),(Eng/Span(SIALANG),#PPLinHousehold(DMDHHSIZ),#Kinds<=5yrs(DMDHHSZA),...,HowOftenDrankMilk,Race(RIDRETH3)_Asian,Race(RIDRETH3)_Black,Race(RIDRETH3)_MexicanAmerican,Race(RIDRETH3)_Other&MultiRacial,Race(RIDRETH3)_OtherHispanic,Race(RIDRETH3)_White,log_vitaminA,female,male
0,1,69,1,1,1,3,0,1,3,0,...,8,0,1,0,0,0,0,4.700480,0,1
1,2,54,0,1,1,3,1,1,4,0,...,9,0,0,0,0,0,1,7.305188,0,1
2,0,72,1,1,1,4,1,1,2,0,...,9,0,0,0,0,0,1,6.484635,0,1
3,9,73,0,1,1,5,1,1,2,0,...,9,0,0,0,0,0,1,6.760415,1,0
4,19,56,1,1,1,4,0,1,1,0,...,9,0,0,1,0,0,0,5.872118,0,1


In [50]:
from sklearn.svm import SVR

In [76]:
target = df.DepressionScore

In [79]:
train = df.iloc[:,2:]

In [81]:
#train.info()

In [82]:
df.to_csv('df.csv', sep=',', encoding='utf-8')